In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import utils
from pathlib import Path

In [ ]:
PATH = '~/Data/holtery_awdowczyk'
fnames = !ls $PATH/*.ISHNE
fnames

In [ ]:
def process(fname):
    file_ext = fname.split('.')[-1]
    if file_ext == 'ecg' or file_ext == 'ISHNE':
        record = utils.load_ishne(fname)
    elif file_ext == 'hea':
        record = utils.load_wfdb(fname)
    else:
        return

    ecg = record['signal']
    sampling_rate = record['sampling_rate']
    left_off = sampling_rate
    right_off = 3*sampling_rate//2
    ecg = utils.clean_ecg(ecg, sampling_rate)
    rpeaks = utils.get_rpeaks(ecg, sampling_rate, left_off, right_off, r_source_lead=0)
    beats = len(rpeaks)
    lead = 0

    image, _ = utils.make_carpet(ecg[lead], rpeaks, first_r=0, beats=beats, left_off=left_off, right_off=right_off)

    return image    

In [ ]:
for fn in fnames:
    print(f'Processing {fn}...')
    im = process(fn)
    if im is None:
        continue
    p1, p2 = np.percentile(im, [1, 99])
    fig, ax = plt.subplots(figsize=(5, 5))
    ax.imshow(im, cmap='jet', vmin=p1, vmax=p2, aspect='auto', interpolation='none')
    im_fname = Path(fn).stem+'.png'
    fig.savefig(f'output/{im_fname}', bbox_inches='tight')
    plt.close(fig)